In [16]:
%%capture
!sudo apt-get update
!sudo apt-get install -y make build-essential libssl-dev zlib1g-dev libbz2-dev \
    libreadline-dev libsqlite3-dev wget curl llvm libncurses5-dev libncursesw5-dev \
    xz-utils tk-dev libffi-dev liblzma-dev git lshw

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libbz2-dev is already the newest version (1.0.8-5build1).
libffi-dev is already the newest version (3.4.2-4).
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
libreadline-dev is already the newest version (8.1.2-1).
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
make is already the newest version (4.3-4.1build1).
tk-dev is already the newest version (8.6.11+1build2).
xz-utils is already the newest version (5.2.5-2ubuntu1).
llv

In [2]:
# %%capture

# -fsSL 表示使用curl 命令下载文件时，不显示进度和错误信息
!curl -fsfL https://ollama.ai/install.sh | sh 

# Go Terminal
# ollama serve
# ollama run deepseek-coder-v2:16b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%####################################                      72.2%
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
%%capture

import subprocess
import sys

# Define the requirements array
requirements = [
    "transformers",
    "langchain",
    "langchain-community",
    "langchain-ollama",
    "termcolor"
]

# Install the packages
for package in requirements:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [13]:
# Model name constant
# MODEL_NAME = "deepseek-coder-v2:16b"
MODEL_NAME = "deepseek-coder-v2:latest"

# Import necessary libraries
from termcolor import cprint

# Initialize Ollama Instance
from langchain_community.llms import Ollama
llm = Ollama(model=MODEL_NAME)

# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model=MODEL_NAME)

In [14]:
# Cell 3: Define a prompt template and generate responses
prompt_template = '''
You are a software engineer tasked with generating summary comment documentation for a given code snippet. Follow these steps:

1. **Review the Summary Policy**: Understand the guidelines for summarizing code.
2. **Analyze the Code Content**: Identify the programming language and functionality of the provided code snippet.
3. **Generate Summary Documentation**: Create a concise summary that describes what the code does, its purpose, and any important details.

### Summary Policy
<SUMMARY_POLICY>
{$SUMMARY_POLICY}
</SUMMARY_POLICY>

### Code Content
<CODE_CONTENT>
{$CODE_CONTENT}
</CODE_CONTENT>

### Output Format
```[lang]
[code with summary]
```
- The entire code block is wrapped in triple backticks (`) to denote it as a code snippet.
- The [lang] specifies the programming language used for clarity and context.
- The [code with summary] section provides both the original code and its documentation.

### Example Outputs
<example1>
Input:
```py
def add(a, b):
    return a + b
```

Output:
```py
def add(a, b):
    """
    Adds two numbers.

    @param a: (int or float) The first number to add.
    @param b: (int or float) The second number to add.
    @return: (int or float) The sum of `a` and `b`.
    """
    return a + b
```
</example 1>

<example2>
Input:
```py
class Library:
    def __init__(self):
        self.books = {}

    def add_book(self, title, author):
        self.books[title] = author

    def checkout_book(self, title):
        if title in self.books:
            del self.books[title]
            return f"You have checked out '{title}'."
        else:
            return f"'{title}' is not available in the library."
```

Output:
```py
class Library:
    """
    A class to manage a collection of books in a library.
    """

    def __init__(self):
        """
        Initializes a new Library instance with an empty collection of books.
        """
        self.books = {}

    def add_book(self, title, author):
        """
        Adds a book to the library.

        @param title: (str) The title of the book to add.
        @param author: (str) The author of the book to add.
        """
        self.books[title] = author

    def checkout_book(self, title):
        """
        Checks out a book from the library.

        @param title: (str) The title of the book to check out.
        @return: (str) A message indicating whether the checkout was successful or if the book is not available.
        """
        if title in self.books:
            del self.books[title]
            return f"You have checked out '{title}'."
        else:
            return f"'{title}' is not available in the library."
```
</example2>
'''

# Predefined inputs
code_examples = [
    """
def fibonacci(n):
    sequence = []
    a, b = 0, 1
    while a < n:
        sequence.append(a)
        a, b = b, a + b
    return sequence
    """,
    """
class Restaurant:
    def __init__(self, name, cuisine_type):
        self.name = name
        self.cuisine_type = cuisine_type
        self.menu = {}

    def add_dish(self, dish_name, price):
        self.menu[dish_name] = price

    def remove_dish(self, dish_name):
        if dish_name in self.menu:
            del self.menu[dish_name]

    def get_menu(self):
        return self.menu
    """,
    """
def merge_sorted_lists(list1, list2):
    merged_list = []
    i, j = 0, 0

    while i < len(list1) and j < len(list2):
        if list1[i] < list2[j]:
            merged_list.append(list1[i])
            i += 1
        else:
            merged_list.append(list2[j])
            j += 1

    merged_list.extend(list1[i:])
    merged_list.extend(list2[j:])

    return merged_list
    """
]




In [15]:
# Predefined inputs
summary_policy = "Summarize code clearly and concisely and functionality."

def generate_response(code_content: str, summary_policy: str) -> str:
    """Generate a response from the model using the prompt template."""
    prompt = prompt_template.replace("{$SUMMARY_POLICY}", summary_policy).replace("{$CODE_CONTENT}", code_content)
    cprint(f"actual_prompt:\n{prompt}", "magenta")
    response = llm(prompt)
    return response
    
# Generate responses for predefined instructions
for code_content in code_examples:
    cprint("=" * 30, "cyan") # Colored separator
    cprint(f"Code Content:\n{code_content}", "yellow")
    output = generate_response(code_content, summary_policy)
    cprint(f"Response from model:\n{output}\n", "green")
    cprint("=" * 30, "cyan") # Colored separator

Code Content:

def fibonacci(n):
    sequence = []
    a, b = 0, 1
    while a < n:
        sequence.append(a)
        a, b = b, a + b
    return sequence
    
actual_prompt:

You are a software engineer tasked with generating summary comment documentation for a given code snippet. Follow these steps:

1. **Review the Summary Policy**: Understand the guidelines for summarizing code.
2. **Analyze the Code Content**: Identify the programming language and functionality of the provided code snippet.
3. **Generate Summary Documentation**: Create a concise summary that describes what the code does, its purpose, and any important details.

### Summary Policy
<SUMMARY_POLICY>
Summarize code clearly and concisely and functionality.
</SUMMARY_POLICY>

### Code Content
<CODE_CONTENT>

def fibonacci(n):
    sequence = []
    a, b = 0, 1
    while a < n:
        sequence.append(a)
        a, b = b, a + b
    return sequence
    
</CODE_CONTENT>

### Output Format
```[lang]
[code with summary]
```
-